# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#!pip freeze > requirements.txt
#!python --version
#!tree ./data 

In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [4]:
!python create_tables.py

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [7]:
def load_multi_data(files):
    data = []
    for fpath in files:
        data.append(pd.read_json(fpath, lines=True))        
    return pd.concat(data, axis=0)

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [8]:
song_files = get_files("./data/song_data/")
song_files[:3]

['/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json']

In [9]:
filepath = song_files[0]
filepath

'/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json'

In [10]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


In [11]:
load_multi_data(song_files[:3])


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
0,ARJIE2Y1187B994AB7,NaN,,NaN,Line Renaud,152.92036,1,SOUPIRU12A6D4FA1E1,Der Kleine Dompfaff,0
0,ARPFHN61187FB575F6,41.88415,"Chicago, IL",-87.63241,Lupe Fiasco,279.97995,1,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),0


## Simple EDA for song_data

In [12]:
df_songs = load_multi_data(song_files)

### 1. Check missing values.

In [13]:
df_songs.isnull().sum(axis=0)

artist_id            0
artist_latitude     41
artist_location      0
artist_longitude    41
artist_name          0
duration             0
num_songs            0
song_id              0
title                0
year                 0
dtype: int64

### 2. Check string column lengh.

In [14]:
df_songs[["artist_id", "artist_location", "artist_name", "song_id", "title"]].apply(lambda x: x.str.len().max())

artist_id          18
artist_location    29
artist_name        94
song_id            18
title              52
dtype: int64

### 3. Check numeric column stats

In [15]:
df_songs.describe()

,artist_latitude,artist_longitude,duration,num_songs,year
count,31.000000,31.000000,72.000000,72.0,72.000000
mean,36.552972,-73.251233,239.440817,1.0,775.041667
std,12.431023,36.058076,105.838061,0.0,978.418751
min,-13.442000,-122.420050,29.544040,1.0,0.000000
25%,34.730385,-92.039350,171.957095,1.0,0.000000
50%,38.899100,-80.112780,227.473835,1.0,0.000000
75%,40.910475,-73.986780,284.401175,1.0,1992.250000
max,56.276090,15.967600,599.248530,1.0,2008.000000


In [16]:
df_songs.year.value_counts()

0       44
2004     4
2005     2
2003     2
2000     2
1997     2
1994     2
1984     1
1961     1
1964     1
1969     1
1972     1
1982     1
2008     1
1985     1
1986     1
1987     1
2007     1
1993     1
1999     1
1992     1
Name: year, dtype: int64

The `0` value of the `year` field seems to represent missing value.

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [17]:
song_data = tuple(df[["song_id", "title", "artist_id", "year", "duration"]].values[0])
song_data

('SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [18]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [19]:
artist_data = tuple(df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0])
artist_data

('ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [20]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [21]:
log_files = get_files("./data/log_data/")
log_files[:3]

['/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-27-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json']

In [22]:
filepath = log_files[0]

In [23]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


## Simple EDA for log_data

In [24]:
df_logs = load_multi_data(log_files)
df_logs[:3]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


### 1. Check missing values.

In [25]:

df_logs.isnull().sum(axis=0)

artist           1236
auth                0
firstName         286
gender            286
itemInSession       0
lastName          286
length           1236
level               0
location          286
method              0
page                0
registration      286
sessionId           0
song             1236
status              0
ts                  0
userAgent         286
userId              0
dtype: int64

### 2. Check string column lengh.

In [26]:
df_logs[["artist", "firstName", "gender", "lastName","level", "location", "song", "userAgent"]].apply(lambda x: x.str.len().max())

artist        89.0
firstName     10.0
gender         1.0
lastName       9.0
level          4.0
location      46.0
song         151.0
userAgent    139.0
dtype: float64

### 3. Check numeric column stats

In [27]:
df_logs.describe()

,itemInSession,length,registration,sessionId,status,ts
count,8056.000000,6820.000000,7.770000e+03,8056.000000,8056.000000,8.056000e+03
mean,21.198858,247.032221,1.540778e+12,598.167577,202.897592,1.542486e+12
std,23.440699,102.975921,2.651572e+08,285.313094,17.994256,7.003166e+08
min,0.000000,15.855870,1.539909e+12,3.000000,200.000000,1.541106e+12
25%,3.000000,197.321998,1.540558e+12,372.000000,200.000000,1.542023e+12
50%,13.000000,232.972605,1.540829e+12,605.000000,200.000000,1.542467e+12
75%,33.000000,274.121992,1.541020e+12,834.000000,200.000000,1.543064e+12
max,127.000000,2594.873020,1.541098e+12,1114.000000,404.000000,1.543608e+12


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [28]:
df = df[df["page"] == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [29]:
t = df["ts"].map(lambda x: pd.Timestamp(x/1000, unit='s'))
t.head()

0   2018-11-09 00:06:17.796000004
1   2018-11-09 00:09:46.796000004
2   2018-11-09 00:12:27.796000004
3   2018-11-09 00:14:52.796000004
4   2018-11-09 00:17:44.796000004
Name: ts, dtype: datetime64[ns]

In [30]:
time_data = [df["ts"].values, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ("start_time", "hour", "day", "weekofyear", "month", "year", "weekday")

In [31]:
time_df = pd.DataFrame({c:d for c, d in zip(column_labels, time_data)})
time_df.head()

,start_time,hour,day,weekofyear,month,year,weekday
0,1541721977796,0,9,45,11,2018,4
1,1541722186796,0,9,45,11,2018,4
2,1541722347796,0,9,45,11,2018,4
3,1541722492796,0,9,45,11,2018,4
4,1541722664796,0,9,45,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [32]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]
user_df.drop_duplicates(subset=["userId"], keep="last", inplace=True)
user_df

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userId,firstName,lastName,gender,level
33,42,Harper,Barrett,M,paid
36,24,Layla,Griffin,F,paid
41,64,Hannah,Calhoun,F,free
48,95,Sara,Johnson,F,paid
50,47,Kimber,Norris,F,free
54,60,Devin,Larson,M,free
57,67,Colm,Santana,M,free
63,55,Martin,Johnson,M,free
84,15,Lily,Koch,F,paid
93,101,Jayden,Fox,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [34]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [35]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row["ts"], row["userId"], row["level"], songid, artistid, row["sessionId"], row["location"], row["userAgent"])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [36]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [37]:
!python etl.py

72 files found in data/song_data
1/72 files processed.
2/72 files processed.
3/72 files processed.
4/72 files processed.
5/72 files processed.
6/72 files processed.
7/72 files processed.
8/72 files processed.
9/72 files processed.
10/72 files processed.
11/72 files processed.
12/72 files processed.
13/72 files processed.
14/72 files processed.
15/72 files processed.
16/72 files processed.
17/72 files processed.
18/72 files processed.
19/72 files processed.
20/72 files processed.
21/72 files processed.
22/72 files processed.
23/72 files processed.
24/72 files processed.
25/72 files processed.
26/72 files processed.
27/72 files processed.
28/72 files processed.
29/72 files processed.
30/72 files processed.
31/72 files processed.
32/72 files processed.
33/72 files processed.
34/72 files processed.
35/72 files processed.
36/72 files processed.
37/72 files processed.
38/72 files processed.
39/72 files processed.
40/72 files processed.
41/72 files processed.
42/72 files processed.
43/72 file